In [1]:
from pyspark import SparkContext
sc = SparkContext()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/18 19:30:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
start_character_id = 5306 #Spider-man
target_character_id = 14 #Adam

In [3]:
hit_counter = sc.accumulator(0)

In [16]:
def convert_to_bfs(line):
    fields = line.split()
    hero_id = int(fields[0])
    conns = []
    
    for con in fields[1:]:
        conns.append(int(con))
        
    color = "WHITE"
    dist = 9999
    
    if(hero_id == start_character_id):
        color = "GRAY"
        dist = 0
        
    return hero_id, (conns, dist, color)

In [14]:
def create_starting_rdd():
    input_file = sc.textFile("Marvel+Graph")
    return input_file.map(convert_to_bfs)

In [29]:
def bfs_map(node):
    char_id = node[0]
    data = node[1]
    
    conns = data[0]
    dist = data[1]
    col = data[2]
    result = []
    
    if col == "GRAY":
        for con in conns:
            new_char_id = con
            new_dist = dist + 1
            new_color = "GRAY"
            
            if(target_character_id == con): 
                hit_counter.add(1)
                
            new_entry = (new_char_id, ([], new_dist, new_color))
            result.append(new_entry)
            
        col = "BLACK"
        
    result.append((char_id, (conns, dist, col)))  
    return result

In [30]:
def bfs_reduce(data1, data2):
    edge_1 = data1[0]
    edge_2 = data2[0]
    
    dist_1 = data1[1]
    dist_2 = data2[1]
    
    color_1 = data1[2]
    color_2 = data2[2]
    
    dist = 9999
    color = color_1
    edges = []
    
    
    if len(edge_1) > 0:
        edges.extend(edge_1)
    
    if len(edge_2) > 0:
        edges.extend(edge_2)
    
    if dist_1 < dist:
        dist = dist_1
    
    if dist_2 < dist:
        dist = dist_2        
        
    if (color_1 == 'WHITE' and (color_2 == 'GRAY' or color_2 == 'BLACK')):
        color = color_2

    if (color_1 == 'GRAY' and color_2 == 'BLACK'):
        color = color_2

    if (color_2 == 'WHITE' and (color_1 == 'GRAY' or color_1 == 'BLACK')):
        color = color_1

    if (color_2 == 'GRAY' and color_1 == 'BLACK'):
        color = color_1
    
    return (edges, dist, color)

---main---

In [31]:
iteration_rdd = create_starting_rdd()
for itr in range(0,10):
    print ("Running BFS itr#" + str(itr+1))
    mapped = iteration_rdd.flatMap(bfs_map)
    
    print ("Processing " + str(mapped.count()) + " values")
    
    if hit_counter.value > 0 :
        print ("Hit target character " + str(hit_counter.value) + " to diff directions" )
        break
    
    iteration_rdd = mapped.reduceByKey(bfs_reduce)

Running BFS itr#1
Processing 8330 values
Running BFS itr#2
Processing 220615 values
Hit target character 1 to diff directions


In [32]:
iteration_rdd = create_starting_rdd()
print(iteration_rdd.count())
mapped = iteration_rdd.flatMap(bfs_map)
print(mapped.count())
mapped.reduceByKey()

6589
8330
